# Example 01
This is the example code from Figure 7 in Peacock, J.R. Kappler, K, Heagy, L., Ronan, T., Kelbert, A., Frassetto, A. (2022) MTH5: an archive and exchangeable data format for magnetotelluric time series data, *Computers & Geoscience*, in review.

This is an example script to read in LEMI424 files for a single station.

The LEMI424 outputs files in hour blocks, so as long as the files are 
continuous all the files can be considered a single run.  Here we have 2 files of 2 minutes each starting on the hour, so 2 runs will be made.

The user will have to input some metadata like station name and run id.

In [1]:
from pathlib import Path
from mth5 import read_file
from mth5.mth5 import MTH5

from mt_metadata.timeseries import Survey, Station

### Set Path to LEMI 424 .TXT files
Example files are supplied in this directory.  They are each 2 minutes long for simplicity

In [2]:
lemi424_dir = Path(r".")
h5_fn = lemi424_dir.joinpath("from_lemi424.mth5")

In [3]:
# remove file if it already exists
if h5_fn.exists():
    h5_fn.unlink()
    print(f"INFO: Removed existing file {h5_fn}")

INFO: Removed existing file from_lemi424.mth5


### Open an MTH5 file

### Add Survey Metadata

In [4]:
survey_metadata = Survey()
survey_metadata.acquired_by.author = "MT Master"
survey_metadata.id = "TEST01"
survey_metadata.fdsn.network = "MT"
survey_metadata.name = "test"

## Fill MTH5

Here we will use context manager `with` to make the MTH5. This will close the file if anything in the process breaks.  

We will add a Station to the Survey Group with Runs for each file 

In [5]:
%timeit

with MTH5() as m:
    m.open_mth5(h5_fn, "w")

    # add survey metadata
    survey_group = m.add_survey(survey_metadata.id, survey_metadata)
    # initialize a station
    station_group = survey_group.stations_group.add_station("mt001")

    # loop over all files in a directory, should be a single station
    for index, lemi424_filename in enumerate(sorted(lemi424_dir.glob("*.TXT"))):
        run_object = read_file(lemi424_filename)
        run_object.run_metadata.id = f"{index:003}"
        # make a run group from first file
        run_group = station_group.add_run(
            run_object.run_metadata.id, run_metadata=run_object.run_metadata
        )
        run_group.from_runts(run_object)
        run_group.update_run_metadata()

    # update station metadata to ensure consistency
    station_group.update_station_metadata()
    survey_group.update_survey_metadata()

    print(m)

2023-09-27T16:21:47.713367-0700 | INFO | mth5.mth5 | _initialize_file | Initialized MTH5 0.2.0 file from_lemi424.mth5 in mode w
/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: TEST01
            ----------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                    |- Group: fap
                    -------------
                    |- Group: fir
                    -------------
                    |- Group: time_delay
                    --------------------
                    |- Group: zpk
                    -------------
                |- Group: Reports
                -----------------
                |- Group: S

### Delete the MTH5 file

In [6]:
h5_fn.unlink()